In [1]:
import requests
from pandas.io.html import read_html
import pandas as pd
import re
import numpy as np
# import only html class
from lxml import html 
from urllib.error import HTTPError

final=['Name','Constellation',
 'Right ascension',
 'Declination',
 'Apparent\xa0magnitude\xa0(V)',
 'Spectral\xa0type',
 'U−B color index',
 'B−V color index',
 'V−R color index',
 'R−I color index',
 'Variable\xa0type',
 'Radial velocity (Rv)',
 'Proper motion (μ)',
 'Parallax (π)',
 'Distance',
 'Absolute\xa0magnitude\xa0(MV)',
 'Mass',
 'Radius',
 'Luminosity',
 'Surface gravity (log\xa0g)',
 'Temperature',
 'Metallicity [Fe/H]',
 'Rotational velocity (v\xa0sin\xa0i)',
 'Age',
 'Period (P)',
 'Semi-major axis (a)',
 'Eccentricity (e)',
 'Inclination (i)',
 'Longitude of the node (Ω)',
 'Periastron epoch (T)',
 'Argument of periastron (ω)(secondary)']


url='https://en.wikipedia.org/wiki/Lists_of_stars_by_constellation'



def get_constellation_links(url):
    const_links=[]
    response = requests.get(url)
    byte_data = response.content
    source_code = html.fromstring(byte_data)
    const_links=[]
    for i in range(1,5):
        path1='//table/tbody/tr[1]/td['+str(i)+']/ul/li/a'
        tree = source_code.xpath(path1)
        for x in range(0,len(tree)):
            x=tree[x].get('href')
            wiki_path="https://en.wikipedia.org"+x
            const_links.append(wiki_path)

    return const_links







In [2]:
#const_links=['https://en.wikipedia.org/wiki/List_of_stars_in_Antlia']
def get_star_links(const_links):
    title_page=[]
    star_links=[]
    title_HIP={}
#url = 'https://en.wikipedia.org/wiki/List_of_stars_in_Antlia'
    for const in const_links:
        col_no=get_HIP_col_no(const)
        path1='//table/tbody/tr'
        i=0
        response = requests.get(const)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        tree = source_code.xpath(path1)
        len(tree)
        for i in range(len(tree)):
            path = '//table/tbody/tr['+str(i)+']/td/a[1]'
            path1='//table/tbody/tr['+str(i)+']/td['+str(col_no)+']'
            tree = source_code.xpath(path)
            if(len(tree)>0):
                star_links.append(tree[0].get('href'))
                name=tree[0].get('title')
                print(name)
                tree_hip=source_code.xpath(path1)
                if(len(tree_hip)>0):
                    HIP=tree_hip[0].text_content()
                    title_HIP[name]=HIP
            xpath_1="//*[@class='firstHeading']"
            #response_title = requests.get(const)
            #byte_data_title = response_title.content
            #source_code_title = html.fromstring(byte_data)
            tree_title = source_code.xpath(xpath_1)
            title=tree_title[0].text_content()  
            title_page.append(title)
    
    return star_links,title_page,title_HIP




In [3]:
def get_HIP_col_no(const):
        path44="//div[@id='mw-content-text']/div/table//th"
        i=0
        response = requests.get(const)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        tree = source_code.xpath(path44)
        count=1
        for i in tree:
            a=i.text_content().strip()
            if a!='HIP':
                count=count+1
            else:
                return count







        

In [4]:
constellation_wiki=get_constellation_links(url)
stars_wiki,page_titles,hip=get_star_links(constellation_wiki)

Alpha Andromedae
Beta Andromedae
Gamma Andromedae
Delta Andromedae
Andromeda Galaxy
51 Andromedae
Omicron Andromedae
Lambda Andromedae
Mu Andromedae
Zeta Andromedae
Upsilon Andromedae
Kappa Andromedae
Phi Andromedae
Iota Andromedae
Pi Andromedae
Epsilon Andromedae
Eta Andromedae
Sigma Andromedae
Nu Andromedae
7 Andromedae
Theta Andromedae
3 Andromedae
65 Andromedae
58 Andromedae
8 Andromedae
Omega Andromedae
Gamma Andromedae
60 Andromedae
Xi Andromedae
Tau Andromedae
HD 10307
Psi Andromedae
22 Andromedae
Chi Andromedae
41 Andromedae
2 Andromedae
V428 Andromedae
Rho Andromedae
HD 2421
64 Andromedae
28 Andromedae
14 Andromedae
49 Andromedae
32 Andromedae
4 Andromedae
6 Persei
62 Andromedae
18 Andromedae
55 Andromedae
11 Andromedae
36 Andromedae
15 Andromedae
63 Andromedae
47 Andromedae
44 Andromedae
5 Andromedae
HD 5788
56 Andromedae
23 Andromedae
HD 16028
13 Andromedae
12 Andromedae
45 Andromedae
HD 14622
10 Andromedae
HD 222109
HD 224635
OU Andromedae
HD 2767
HD 1606
KK Andromedae
6 An

WX Arietis (page does not exist)
Teegarden's star
XY Arietis (page does not exist)
PSR B0301+19 (page does not exist)
Capella (star)
Beta Aurigae
Theta Aurigae
Iota Aurigae
Epsilon Aurigae
Eta Aurigae
Zeta Aurigae
Delta Aurigae
Nu Aurigae
Pi Aurigae
Kappa Aurigae
Tau Aurigae
16 Aurigae
Lambda Aurigae
Chi Aurigae
Upsilon Aurigae
2 Aurigae
Psi2 Aurigae
Mu Aurigae
63 Aurigae
Psi1 Aurigae
Omega Aurigae
Xi Aurigae
9 Aurigae
Psi7 Aurigae
14 Aurigae
Sigma Aurigae
Psi4 Aurigae
19 Aurigae
Phi Aurigae
65 Aurigae
Rho Aurigae
Psi6 Aurigae
66 Aurigae
Psi5 Aurigae
49 Aurigae
45 Aurigae
Psi3 Aurigae
40 Aurigae
IQ Aurigae
26 Aurigae
UU Aurigae
Omicron Aurigae
PU Aurigae
HD 34790
51 Aurigae
36 Aurigae
RT Aurigae
53 Aurigae
41 Aurigae
WW Aurigae
HD 30453
Psi9 Aurigae
64 Aurigae
47 Aurigae
39 Aurigae
5 Aurigae
V433 Aurigae
AE Aurigae
HD 41429
54 Aurigae
62 Aurigae
HIP 26712
38 Aurigae
41 Aurigae
59 Aurigae
AR Aurigae
V352 Aurigae
HD 35519
V538 Aurigae
NO Aurigae
V440 Aurigae (page does not exist)
60 Auri

Z Camelopardalis variable
Messier 67
RR Lyrae variable
Flare star
Red dwarf
G 9-38
YBP 1194
DX Cancri
SU Ursae Majoris variable
WZ Sagittae variable
Dwarf nova
AZ Cancri
RX J0806.3+1527
Pulsar
Cor Caroli
Beta Canum Venaticorum
24 Canum Venaticorum
20 Canum Venaticorum
AW Canum Venaticorum
5 Canum Venaticorum
25 Canum Venaticorum
HR 5110
HD 115004
6 Canum Venaticorum
21 Canum Venaticorum
14 Canum Venaticorum (page does not exist)
3 Canum Venaticorum (page does not exist)
HD 109317 (page does not exist)
La Superba
HD 120164 (page does not exist)
23 Canum Venaticorum (page does not exist)
Alpha1 Canum Venaticorum
HD 120420 (page does not exist)
HD 113847 (page does not exist)
2 Canum Venaticorum (page does not exist)
TU Canum Venaticorum (page does not exist)
19 Canum Venaticorum
HD 115723 (page does not exist)
HD 111604 (page does not exist)
HD 116957 (page does not exist)
HD 120047 (page does not exist)
HD 120819 (page does not exist)
17 Canum Venaticorum (page does not exist)
HD 121164

Delta Capricorni
Beta Capricorni
Alpha² Capricorni
Gamma Capricorni
Zeta Capricorni
Theta Capricorni
Omega Capricorni
Psi Capricorni
Iota Capricorni
Alpha1 Capricorni
24 Capricorni
36 Capricorni
Epsilon Capricorni
Kappa Capricorni
Nu Capricorni
Rho Capricorni
Eta Capricorni
Mu Capricorni
Pi Capricorni
46 Capricorni
Upsilon Capricorni
42 Capricorni
Phi Capricorni
Tau2 Capricorni
41 Capricorni
Sigma Capricorni
Chi Capricorni
29 Capricorni (page does not exist)
33 Capricorni (page does not exist)
30 Capricorni (page does not exist)
Lambda Capricorni
HD 203475 (page does not exist)
HD 195564
HD 199260 (page does not exist)
37 Capricorni (page does not exist)
HD 192310
19 Capricorni (page does not exist)
35 Capricorni (page does not exist)
HD 196857 (page does not exist)
Xi2 Capricorni
4 Capricorni (page does not exist)
HD 198174 (page does not exist)
44 Capricorni (page does not exist)
HD 199443 (page does not exist)
17 Capricorni (page does not exist)
Omicron Capricorni
45 Capricorni (pag

HD 93695 (page does not exist)
HD 95122 (page does not exist)
HD 92399 (page does not exist)
HD 96248 (page does not exist)
HD 57852/57853 (page does not exist)
QX Carinae (page does not exist)
HD 96446 (page does not exist)
HD 53143
VY Carinae (page does not exist)
AG Carinae
BO Carinae
HD 93403
HD 93129A
V Carinae (page does not exist)
HD 93843 (page does not exist)
HD 95086
HD 93250
HR Carinae
WR 40 (page does not exist)
HD 93205
IW Carinae (page does not exist)
HD 65216
HD 97152 (page does not exist)
HD 63765
Y Carinae
HD 51608 (page does not exist)
HD 85628 (page does not exist)
YZ Carinae (page does not exist)
WR 16 (page does not exist)
EM Carinae (page does not exist)
RT Carinae
HD 85567 (page does not exist)
UX Carinae (page does not exist)
XZ Carinae (page does not exist)
GG Carinae (page does not exist)
V403 Carinae (page does not exist)
Tr 16-104 (page does not exist)
V348 Carinae (page does not exist)
WR 25
HD 60435 (page does not exist)
WR 23 (page does not exist)
HD 9202

...

HD 155259 (page does not exist)
12 Scorpii
HD 159176 (page does not exist)
11 Scorpii (page does not exist)
HD 156293 (page does not exist)
HD 152408
HD 148247 (page does not exist)
HD 147723 (page does not exist)
V1003 Scorpii (page does not exist)
HD 142883 (page does not exist)
HD 150331 (page does not exist)
HD 152293 (page does not exist)
HD 158799 (page does not exist)
HD 162391 (page does not exist)
HD 145191 (page does not exist)
HD 144844 (page does not exist)
3 Scorpii (page does not exist)
HD 160928 (page does not exist)
V957 Scorpii
V929 Scorpii (page does not exist)
HD 145838 (page does not exist)
V923 Scorpii (page does not exist)
Gliese 667
HD 157097 (page does not exist)
HD 149886 (page does not exist)
HD 154783 (page does not exist)
HD 162189 (page does not exist)
V1068 Scorpii (page does not exist)
HD 153368 (page does not exist)
HD 155450 (page does not exist)
HD 155826 (page does not exist)
HD 146850 (page does not exist)
HD 154310 (page does not exist)
HD 158105 (p

Alpha Serpentis
Eta Serpentis
Mu Serpentis
Xi Serpentis
Beta Serpentis
Epsilon Serpentis
Gamma Serpentis
Kappa Serpentis
Delta Serpentis
Omicron Serpentis
Nu Serpentis
Lambda Serpentis
Iota Serpentis
Zeta Serpentis
Theta1 Serpentis
Rho Serpentis
Pi Serpentis
Sigma Serpentis
Theta2 Serpentis
5 Serpentis
B Serpentis
10 Serpentis
Tau1 Serpentis
R Serpentis
D Serpentis
D Serpentis
Delta Serpentis
Omega Serpentis
16 Serpentis
3 Serpentis
Chi Serpentis
6 Serpentis
C Serpentis
A2 Serpentis
QY Serpentis (page does not exist)
A1 Serpentis
Phi Serpentis
64 Serpentis
4 Serpentis (page does not exist)
45 Serpentis (page does not exist)
47 Serpentis (page does not exist)
Upsilon Serpentis
E Serpentis
Tau7 Serpentis
3 Herculis (page does not exist)
Psi Serpentis
FL Serpentis (page does not exist)
Tau5 Serpentis
61 Serpentis (page does not exist)
Tau6 Serpentis
39 Serpentis (page does not exist)
43 Serpentis (page does not exist)
Tau3 Serpentis
8 Serpentis (page does not exist)
NW Serpentis (page doe

Beta Trianguli
Alpha Trianguli
Gamma Trianguli
Delta Trianguli
Iota Trianguli
14 Trianguli (page does not exist)
7 Trianguli (page does not exist)
10 Trianguli (page does not exist)
12 Trianguli (page does not exist)
15 Trianguli (page does not exist)
R Trianguli (page does not exist)
Epsilon Trianguli
11 Trianguli (page does not exist)
HD 10390 (page does not exist)
HD 11007 (page does not exist)
HD 11928 (page does not exist)
HD 15755 (page does not exist)
13 Trianguli
HD 10348 (page does not exist)
HD 16187 (page does not exist)
HD 16220 (page does not exist)
5 Trianguli (page does not exist)
HD 15464 (page does not exist)
HD 10588 (page does not exist)
HD 13747 (page does not exist)
HD 9298 (page does not exist)
1 Trianguli
HD 13189
HD 9446
HD 12545
X Trianguli (page does not exist)
V Trianguli (page does not exist)
RV Trianguli (page does not exist)
WASP-56
RW Trianguli (page does not exist)
HAT-P-38 (page does not exist)
AI Trianguli (page does not exist)
Var 83
Var B (star) (pag

HD 85563 (page does not exist)
NN Velorum (page does not exist)
HD 89736 (page does not exist)
HD 74167 (page does not exist)
HD 96113 (page does not exist)
HD 72737 (page does not exist)
HD 76653
HD 75276 (page does not exist)
HD 86087 (page does not exist)
HD 85483 (page does not exist)
HD 85980 (page does not exist)
HD 81136 (page does not exist)
HD 91355 (page does not exist)
AH Velorum
HD 79275 (page does not exist)
HD 88842 (page does not exist)
HR 3413 (page does not exist)
HD 76110 (page does not exist)
HD 89569 (page does not exist)
HD 95347 (page does not exist)
HD 72067 (page does not exist)
HD 68657 (page does not exist)
LR Velorum (page does not exist)
HD 79694 (page does not exist)
HD 79807 (page does not exist)
HD 69302 (page does not exist)
HD 77020 (page does not exist)
HD 71043 (page does not exist)
HD 74273 (page does not exist)
IW Velorum (page does not exist)
HD 91437 (page does not exist)
HD 72832 (page does not exist)
HD 76161 (page does not exist)
HD 79621 (page

54 Virginis (page does not exist)
134 G. Virginis (page does not exist)
85 G. Virginis (page does not exist)
213 G. Virginis (page does not exist)
20 Virginis (page does not exist)
208 G. Virginis (page does not exist)
IQ Virginis (page does not exist)
EP Virginis (page does not exist)
26 G. Virginis (page does not exist)
112 G. Virginis (page does not exist)
207 G. Virginis (page does not exist)
68 G. Virginis (page does not exist)
98 G. Virginis (page does not exist)
185 G. Virginis (page does not exist)
210 G. Virginis (page does not exist)
51 G. Virginis (page does not exist)
129 G. Virginis (page does not exist)
204 G. Virginis (page does not exist)
225 G. Virginis (page does not exist)
33 G. Virginis (page does not exist)
29 G. Virginis (page does not exist)
211 G. Virginis (page does not exist)
FS Virginis (page does not exist)
35 Virginis (page does not exist)
150 G. Virginis (page does not exist)
221 G. Virginis (page does not exist)
240 G. Virginis (page does not exist)
141 G

In [7]:
hip

{'Alpha Andromedae': '677',
 'Beta Andromedae': '5447',
 'Gamma Andromedae': '',
 'Delta Andromedae': '3092',
 'Andromeda Galaxy': '',
 '51 Andromedae': '7607',
 'Omicron Andromedae': '113726',
 'Lambda Andromedae': '116584',
 'Mu Andromedae': '4436',
 'Zeta Andromedae': '3693',
 'Upsilon Andromedae': '7513',
 'Kappa Andromedae': '116805',
 'Phi Andromedae': '5434',
 'Iota Andromedae': '116631',
 'Pi Andromedae': '2912',
 'Epsilon Andromedae': '3031',
 'Eta Andromedae': '4463',
 'Sigma Andromedae': '1473',
 'Nu Andromedae': '3881',
 '7 Andromedae': '114570',
 'Theta Andromedae': '1366',
 '3 Andromedae': '113919',
 '65 Andromedae': '11313',
 '58 Andromedae': '9977',
 '8 Andromedae': '115022',
 'Omega Andromedae': '6813',
 '60 Andromedae': '10340',
 'Xi Andromedae': '6411',
 'Tau Andromedae': '7818',
 'HD 10307': '7918',
 'Psi Andromedae': '117221',
 '22 Andromedae': '841',
 'Chi Andromedae': '7719',
 '41 Andromedae': '5317',
 '2 Andromedae': '113788',
 'V428 Andromedae': '2900',
 'Rho A

In [9]:
stars=[]
for i in stars_wiki:
    wiki_path="https://en.wikipedia.org"+i
    stars.append(wiki_path)
stars

['https://en.wikipedia.org/wiki/Alpha_Andromedae',
 'https://en.wikipedia.org/wiki/Beta_Andromedae',
 'https://en.wikipedia.org/wiki/Gamma_Andromedae',
 'https://en.wikipedia.org/wiki/Delta_Andromedae',
 'https://en.wikipedia.org/wiki/Andromeda_Galaxy',
 'https://en.wikipedia.org/wiki/51_Andromedae',
 'https://en.wikipedia.org/wiki/Omicron_Andromedae',
 'https://en.wikipedia.org/wiki/Lambda_Andromedae',
 'https://en.wikipedia.org/wiki/Mu_Andromedae',
 'https://en.wikipedia.org/wiki/Zeta_Andromedae',
 'https://en.wikipedia.org/wiki/Upsilon_Andromedae',
 'https://en.wikipedia.org/wiki/Kappa_Andromedae',
 'https://en.wikipedia.org/wiki/Phi_Andromedae',
 'https://en.wikipedia.org/wiki/Iota_Andromedae',
 'https://en.wikipedia.org/wiki/Pi_Andromedae',
 'https://en.wikipedia.org/wiki/Epsilon_Andromedae',
 'https://en.wikipedia.org/wiki/Eta_Andromedae',
 'https://en.wikipedia.org/wiki/Sigma_Andromedae',
 'https://en.wikipedia.org/wiki/Nu_Andromedae',
 'https://en.wikipedia.org/wiki/7_Andromeda

In [24]:
len(stars)

9365

In [30]:
Sample1=stars[0:500]

In [34]:
Sample2=stars[500:1200]

In [37]:
Sample3=stars[1200:2000]

In [39]:
Sample4=stars[2000:2800]

In [41]:
Sample5=stars[2800:3600]

In [43]:
Sample6=stars[3600:4400]

In [45]:
Sample7=stars[4400:5200]

In [51]:
Sample8=stars[5200:6000]

In [53]:
Sample9=stars[6000:6800]

In [56]:
Sample10=stars[6800:7600]

In [64]:
Sample11=stars[7600:8400]

In [70]:
Sample12=stars[8400:]
len(Sample12)

965

In [3]:
df=pd.DataFrame(columns=final)
df

,Name,Constellation,Right ascension,Declination,Apparent magnitude (V),Spectral type,U−B color index,B−V color index,V−R color index,R−I color index,...,Metallicity [Fe/H],Rotational velocity (v sin i),Age,Period (P),Semi-major axis (a),Eccentricity (e),Inclination (i),Longitude of the node (Ω),Periastron epoch (T),Argument of periastron (ω)(secondary)


In [20]:
def get_star_details(stars,df):
    for links in stars:
        response = requests.get(links)
        byte_data = response.content
        source_code = html.fromstring(byte_data)
        xpath="//*[@class='firstHeading']"
# jump to preferred html element
        tree = source_code.xpath(xpath)
        print(tree[0].text_content())
        if(tree[0].text_content() in page_titles):
            continue
        try:    
            infoboxes = read_html(links, index_col=0,attrs={"class":"infobox"})
        except (ValueError,NameError,HTTPError):
            continue
        
        list_of_values=[]

        for x in final:
            try: 
                z=infoboxes[0].xs(x).values[0]
                if isinstance(z,(str)):
                    y=z.encode("utf-8","ignore")
                    y=y.decode()
                    list_of_values.append(y)
                elif isinstance(z,(np.ndarray)):
                    if(len(z)>=1):
                        a=z[0]
                        a=a.encode("utf-8","ignore")
                        a=a.decode()
                        list_of_values.append(a)
                else:
                    list_of_values.append("")
            except (KeyError,NameError):
                list_of_values.append("")
        list_of_values[0]=tree[0].text_content()
        final_val=[]
        for x in list_of_values:
            t=x.replace('\xa0',' ')
            t=re.sub(r'\[\d+\]', '', t)
            final_val.append(t)
        f_series=pd.Series(final_val,index=df.columns)
        df=df.append(f_series,ignore_index=True)
        
    return df
    

In [32]:
stars_data_1=get_star_details(Sample1,df)


Alpha Andromedae
Beta Andromedae
Gamma Andromedae
Delta Andromedae
Andromeda Galaxy
51 Andromedae
Omicron Andromedae
Lambda Andromedae
Mu Andromedae
Zeta Andromedae
Upsilon Andromedae
Kappa Andromedae
Phi Andromedae
Iota Andromedae
Pi Andromedae
Epsilon Andromedae
Eta Andromedae
Sigma Andromedae
Nu Andromedae
7 Andromedae
Theta Andromedae
3 Andromedae
65 Andromedae
58 Andromedae
8 Andromedae
Omega Andromedae
Gamma Andromedae
60 Andromedae
Xi Andromedae
Tau Andromedae
HD 10307
Psi Andromedae
22 Andromedae
Chi Andromedae
41 Andromedae
2 Andromedae
HD 3346
Rho Andromedae
HD 2421
64 Andromedae
28 Andromedae
14 Andromedae
49 Andromedae
32 Andromedae
4 Andromedae
6 Persei
62 Andromedae
18 Andromedae
55 Andromedae
11 Andromedae
36 Andromedae
15 Andromedae
63 Andromedae
47 Andromedae
44 Andromedae
5 Andromedae
HD 5789/5788
56 Andromedae
23 Andromedae
HD 16028
13 Andromedae
12 Andromedae
45 Andromedae
HD 14622
10 Andromedae
HD 222109
HD 224635
OU Andromedae
HD 2767
HD 1606
KK Andromedae
6 Andro

In [36]:
stars_data_2=get_star_details(Sample2,df)

List of stars in Ara
List of stars in Ara
HD 153053
HD 150136
V539 Arae
List of stars in Ara
HD 157753
List of stars in Ara
List of stars in Ara
List of stars in Ara
List of stars in Ara
List of stars in Ara
HD 156768
List of stars in Ara
List of stars in Ara
HD 157662
HD 151967
HD 156091
HD 157819
List of stars in Ara
HD 155035
HD 158220
HD 150576
HD 153221
HD 152082
HD 158476
HD 153791
HR 6384
List of stars in Ara
List of stars in Ara
Nu2 Arae
List of stars in Ara
HD 165493
List of stars in Ara
HD 149837
List of stars in Ara
List of stars in Ara
List of stars in Ara
HD 153261
List of stars in Ara
List of stars in Ara
HD 156331
HD 160342
HD 149989
Rho1 Arae
List of stars in Ara
List of stars in Ara
HD 153201
HD 153370
HD 151566
List of stars in Ara
List of stars in Ara
List of stars in Ara
List of stars in Ara
HD 157832
HD 156411
R Arae
HD 155555
HD 156385
HD 154857
V535 Arae
HD 154672
RW Arae
HD 152079
Gliese 674
Gliese 676
S Arae
Stingray Nebula
AE Arae
CPD -56 8032
AT Arae
GX 339-4

Beta Canum Venaticorum
24 Canum Venaticorum
20 Canum Venaticorum
AW Canum Venaticorum
5 Canum Venaticorum
25 Canum Venaticorum
HR 5110
HD 115004
6 Canum Venaticorum


In [38]:
stars_data_3=get_star_details(Sample3,df)

21 Canum Venaticorum
14 Canum Venaticorum
3 Canum Venaticorum
HD 109317
La Superba
HD 120164
23 Canum Venaticorum
Cor Caroli
HD 120420
HD 113847
2 Canum Venaticorum
TU Canum Venaticorum
19 Canum Venaticorum
HD 115723
HD 111604
HD 116957
HD 120047
HD 120819
17 Canum Venaticorum
HD 121164
HD 120048
10 Canum Venaticorum
HD 119458
HD 114357
HD 115810
4 Canum Venaticorum
HD 117710
CL Canum Venaticorum
HD 112570
7 Canum Venaticorum
HD 119035
HD 119081
HD 109345
11 Canum Venaticorum
HD 112171
15 Canum Venaticorum
HD 122675
HD 119445
HD 121682
HD 107610
HD 110834
HD 116303
9 Canum Venaticorum
HD 118156
HD 110066
HD 117361
HD 120600
HD 121197
HD 117261
HD 111572
HD 114975
V Canum Venaticorum
HD 108100
BM Canum Venaticorum
R Canum Venaticorum
HD 109995
HD 115781
RS Canum Venaticorum
TT Canum Venaticorum
HD 115444
TX Canum Venaticorum
VZ Canum Venaticorum
HIP 65407
Gliese 521
BI Canum Venaticorum
W Canum Venaticorum
Gliese 490
ST Canum Venaticorum
UZ Canum Venaticorum
Z Canum Venaticorum
DG Canum

OGLE-TR-113
2S 0921-630
OGLE-TR-123
OGLE-TR-111
OGLE-TR-132
OY Carinae
OGLE-TR-182
OGLE-TR-211
OGLE-TR-122
OGLE2-TR-L9
AXP 1E 1048-59
DENIS J081730.0−615520
PSR J1048-5832
PSR J1119-6127
Alpha Cassiopeiae
Beta Cassiopeiae
Gamma Cassiopeiae
Delta Cassiopeiae
Epsilon Cassiopeiae
Eta Cassiopeiae
Zeta Cassiopeiae
50 Cassiopeiae
Kappa Cassiopeiae
Theta Cassiopeiae
Iota Cassiopeiae
Omicron Cassiopeiae
48 Cassiopeiae
Rho Cassiopeiae
Upsilon2 Cassiopeiae
Chi Cassiopeiae
Psi Cassiopeiae
Lambda Cassiopeiae
Xi Cassiopeiae
HR 244
R Cassiopeiae
Upsilon1 Cassiopeiae
1 Cassiopeiae
HD 19275
Tau Cassiopeiae
Sigma Cassiopeiae
AR Cassiopeiae
Nu Cassiopeiae
Pi Cassiopeiae
Phi Cassiopeiae
4 Cassiopeiae
Omega Cassiopeiae
HD 3240
V509 Cassiopeiae
Mu Cassiopeiae
HD 15920
42 Cassiopeiae
49 Cassiopeiae
47 Cassiopeiae
40 Cassiopeiae
HD 11946
31 Cassiopeiae
HD 4775
12 Cassiopeiae
HD 4222
23 Cassiopeiae
6 Cassiopeiae
HD 3474
HD 9900
HD 223173
HD 5408
HD 6960
HD 220369
10 Cassiopeiae
32 Cassiopeiae
43 Cassiopeiae
H

In [40]:
stars_data_4=get_star_details(Sample4,df)

HD 225289
HD 16024
38 Cassiopeiae
HD 3856
HR 297
HD 1279
HD 4440
HD 225009
V762 Cassiopeiae
HD 19065
9 Cassiopeiae
HD 6130
HD 2952
HD 2626
SU Cassiopeiae
HD 16769
HR 273
HR 5
HD 6211
52 Cassiopeiae
HD 11857
V373 Cassiopeiae
HD 4817
55 Cassiopeiae
AO Cassiopeiae
HD 12173
HD 9030
HD 17958
HD 224784
HD 2589
13 Cassiopeiae
HD 3924
ADS 1359
HD 5343
HD 224870
HD 4881
HD 225094
HD 217673
HD 222618
HD 10587
HD 13222
HD 21970
RZ Cassiopeiae
HD 222570
HD 5128
HD 5273
HD 6540
HD 7732
35 Cassiopeiae
HD 10362
HD 223386
HD 8424
HD 10293
HD 8272
HD 223421
HD 7925
V805 Cassiopeiae
HD 4818
HD 4295
HD 5357
V465 Cassiopeiae
HD 220074
HD 5715
V761 Cassiopeiae
HD 222682
HD 371
HD 2904
HD 4362
HD 6497
HD 218440
HD 220130
HR 9017
HD 1142
HD 217944
HD 5459
HD 222748
HD 1601
HD 17581
HD 222932
16 Cassiopeiae
V801 Cassiopeiae
HD 223552
HD 224404
HD 5927
HD 6028
HD 9811
HR 8770
HD 19439
V649 Cassiopeiae
54 Cassiopeiae
V784 Cassiopeiae
HD 11755
TU Cassiopeiae
HD 223960
V Cassiopeiae
HD 14817
HD 17505
CC Cassiopei

HD 123377
HD 123492
Zeta Circini
HD 129462
BU Circini
HD 121557
HD 128917
HD 130227
HD 133792
HD 134468
HD 134060
HD 125990
HD 134657
HD 130942
HD 129092
HD 131491
HD 133557
HD 130021
BP Circini
HD 129445
WR 70
List of most luminous stars
BX Circini
BW Circini
Circinus X-1
PSR B1451-68
PSR B1509−58
Alpha Columbae
Beta Columbae
Delta Columbae
Epsilon Columbae
Eta Columbae
Gamma Columbae
Kappa Columbae
Omicron Columbae
Lambda Columbae
Xi Columbae
Theta Columbae
Mu Columbae
HD 46568
Nu2 Columbae
WZ Columbae
HD 46815
HD 37811
HD 36848
Pi2 Columbae
Sigma Columbae
HD 41047
AF Columbae
HD 43899
HD 40091


In [42]:
stars_data_5=get_star_details(Sample5,df)

HD 44506
HD 36187
HD 47144
HD 39720
HD 45145
HD 41534
SW Columbae
HD 47500
HD 34266
HD 36874
HD 37192
HD 44323
HD 41843
HD 41759
HD 37717
HD 42054
HD 46349
HD 36060
HD 43940
HD 34868
HD 43847
HD 47463
AN Columbae
Nu1 Columbae
Pi1 Columbae
HD 38138
HD 46365
HD 38804
HD 45383
HD 38385
HD 37286
SX Columbae
HD 38885
HD 45306
HD 45983
HD 35046
HD 39891
HD 38056
HD 47230
HD 47475
HD 40359
HD 46727
HD 39543
HD 45680
HD 39901
HD 43848
CD-38 2551
L 449-1
TV Columbae
AP Columbae
WASP-160
TX Columbae
4U 0513-40
Beta Comae Berenices
Alpha Comae Berenices
Gamma Comae Berenices
11 Comae Berenices
36 Comae Berenices
12 Comae Berenices
23 Comae Berenices
41 Comae Berenices
35 Comae Berenices
37 Comae Berenices
14 Comae Berenices
7 Comae Berenices
31 Comae Berenices
16 Comae Berenices
HD 106760
24 Comae Berenices
6 Comae Berenices
27 Comae Berenices
13 Comae Berenices
Alpha Comae Berenices
17 Comae Berenices
21 Comae Berenices
18 Comae Berenices
26 Comae Berenices
HD 107325
FS Comae Berenices
5 Comae B

15 Delphini
1 Delphini
HD 199223
HD 194012
HD 193556
HD 193373
HD 194953
HD 198404
HD 195479
EU Delphini
HD 194688
HD 194937
LU Delphini
HD 201196
HD 194526
14 Delphini
HD 200430
HD 194578
HD 198070
HD 196885
HD 195909
HD 197076
HD 194841
HD 194616
U Delphini
HD 195019
Gliese 795
V Delphini
DM Delphini
LS Delphini
MR Delphini
TX Delphini
DX Delphini
W Delphini
TY Delphini
HAT-P-23
WASP-2
BX Delphini
WASP-176
HR Delphini
HU Delphini
He 2-467
CM Delphini
V339 Delphini
WISEPC J205628.90+145953.3
Alpha Doradus
Beta Doradus
Gamma Doradus
Delta Doradus
HD 40409
Zeta Doradus
Theta Doradus
Eta2 Doradus
Nu Doradus
Epsilon Doradus
Lambda Doradus
WZ Doradus
Kappa Doradus
HD 37297
Pi2 Doradus
Pi1 Doradus
R Doradus
Eta1 Doradus
HD 36584
HD 27604
HD 31746
HD 36876
HD 28255
HR 1960
HD 46730
HD 37501
HD 39963
HD 31754
HD 24863
HD 30610
XY Doradus
XZ Doradus
AB Doradus
HD 28254
HD 30177
HD 33579
S Doradus
WASP-62
HD 38282
HD 37974
Melnick 42
AA Doradus
RW Doradus
R99
HD 36402
HV 883
HD 34664
Gliese 163

In [44]:
stars_data_6=get_star_details(Sample6,df)

PSR B0540-69
PSR J0537−6910
SGR 0525−66
Gamma Draconis
Eta Draconis
Beta Draconis
Delta Draconis
Zeta Draconis
Iota Draconis
Chi Draconis
Thuban
Xi Draconis
Lambda Draconis
Epsilon Draconis
Kappa Draconis
Theta Draconis
Phi Draconis
HD 81817
Tau Draconis
Rho Draconis
Psi1 Draconis
10 Draconis
Pi Draconis
Omicron Draconis
Sigma Draconis
Omega Draconis
45 Draconis
42 Draconis
Upsilon Draconis
18 Draconis
HD 151613
HD 91190
Nu Draconis
19 Draconis
Nu Draconis
Mu Draconis
HD 175535
15 Draconis
6 Draconis
CL Draconis
39 Draconis
36 Draconis
Cataclysmic variable star
64 Draconis
8 Draconis
26 Draconis
7 Draconis
Psi2 Draconis
68 Draconis
Binary star
Alruba
Mu Draconis
Psi1 Draconis
Be star
HD 193664
HD 139357
HD 113337
Carbon star
Semiregular variable star
BY Draconis
HD 156279
HD 163607 b
HD 163607 b
Mira variable
Struve 2398
Transit (astronomy)
HD 191939
Struve 2398
Kepler-10
GSC 03549-02811
Kepler-447b
Kepler-4
Kepler-90
GD 356
Alpha Equulei
Delta Equulei
Gamma Equulei
Beta Equulei
Epsilo

Gliese 623
WASP-86
HAT-P-67
GSC 02620-00648
WASP-103
WASP-148
GSC 03089-00929
HAT-P-18
WASP-92
HAT-P-55
WASP-135
Alpha Horologii
R Horologii
Delta Horologii
Beta Horologii
Mu Horologii
Zeta Horologii
Nu Horologii
Eta Horologii
HD 27588
Lambda Horologii
Iota Horologii
HD 22231
TW Horologii
HD 23719
Gamma Horologii
HD 14641
HD 20640
HD 24706
TU Horologii
HD 18185
HD 19948
HD 17254
HD 18265
HD 17326
HD 16170
HD 15646
HD 26927
TZ Horologii
HD 21011
HD 18292
HD 23670
HD 21360
HD 23308
HD 27631
WASP-120
Gliese 1061
WW Horologii
Alphard
Gamma Hydrae
Zeta Hydrae
Nu Hydrae
Pi Hydrae
Epsilon Hydrae
Xi Hydrae
Lambda Hydrae
Mu Hydrae
Theta Hydrae
Iota Hydrae
30 Monocerotis
Upsilon1 Hydrae
Delta Hydrae
Beta Hydrae
Eta Hydrae
12 Hydrae
Rho Hydrae
58 Hydrae
Sigma Hydrae
Tau Hydrae
Tau Hydrae
Upsilon2 Hydrae
F Hydrae
2 Sextantis
Omicron Hydrae
HD 81799
HD 83953
26 Hydrae
51 Hydrae
27 Hydrae
9 Hydrae
HD 85859
HD 92036
U Hydrae
Phi Hydrae
Chi1 Hydrae
HD 84117
17 Crateris
HD 85951
Psi Hydrae
52 Hydrae
6 

In [46]:
stars_data_7=get_star_details(Sample7,df)

HR 4339
HD 72561
HR 4458
HD 76151
Phi2 Hydrae
Algol variable
Be star
HD 100307
R Hydrae
HD 96700
HD 82943
HD 90156
V Hydrae
HD 86264
HD 72659
HD 74156
Phi1 Hydrae
HD 86226
HD 70573
Gliese 433
Transit (astronomy)
Gliese 357
TW Hydrae
WASP-25
WASP-36
Gliese 3634
Beta Hydri
Alpha Hydri
Gamma Hydri
Delta Hydri
Epsilon Hydri
Eta2 Hydri
Nu Hydri
Zeta Hydri
Lambda Hydri
Mu Hydri
Theta Hydri
Iota Hydri
Pi1 Hydri
Pi2 Hydri
BN Hydri
HD 10615
HD 8810
HD 1801
HD 21722
Kappa Hydri
HD 9896
Tau2 Hydri
HD 11995
HD 12477
HD 10042
HD 19940
HD 21563
Sigma Hydri
HD 10052
CT Hydri
Tau1 Hydri
HD 12270
HD 11944
HD 1221
HD 1237
CN Hydri
Eta1 Hydri
HD 10180
HD 20003
HD 19918
HD 2685
WASP-126
VW Hydri
SMSS J031300.36−670839.3
WX Hydri
BL Hydri
Alpha Indi
Beta Indi
Theta Indi
Delta Indi
Eta Indi
Epsilon Indi
Zeta Indi
Iota Indi
Mu Indi
Nu Indi
Omicron Indi
HD 217831
HD 212728
HD 207229
Kappa2 Indi
HD 202103
HD 199623
HD 212211
HD 207964
T Indi
HD 206399
HD 208796
Rho Indi
Gamma Indi
BG Indi
HD 219644
Pi Indi
HD 

HD 67224
HD 55866
HD 60294
HD 60652
8 Lyncis
HD 80024
HD 78366
HD 75523
HD 75556
19 Ursae Majoris
HD 70647
HD 50551
HD 63332
4 Lyncis
HD 70771
23 Lyncis
HD 80441
CY Lyncis
HD 52859
HD 71906
32 Lyncis
HD 76944
5 Leonis Minoris
HD 64491
HD 71952
25 Lyncis
HD 67370
HD 69149
HD 47979
HD 71148
HD 60654
HD 65801
28 Lyncis
HD 63586
HD 64958
HD 52860
CC Lyncis
HD 77093
HD 52708
HD 59826
HD 58681
HD 59721
7 Lyncis
HD 44061
HD 67539
54 Camelopardalis
HD 63588
HD 63630
HD 65392
HD 69682
17 Lyncis


In [52]:
stars_data_8=get_star_details(Sample8,df)

52 Camelopardalis
HD 71866
46 Camelopardalis
HD 82780
19 Lyncis
Y Lyncis
3 Lyncis
48 Camelopardalis
R Lyncis
44 Camelopardalis
45 Camelopardalis
HD 80715
20 Lyncis
HD 75898
T Lyncis
BE Lyncis
SZ Lyncis
TT Lyncis
UV Lyncis
SW Lyncis
WASP-13
AN Lyncis
XO-4
IRC +60169
14 Lyncis
XO-2 (star)
U Lyncis
TV Lyncis
UU Lyncis
XO-5
BK Lyncis
BH Lyncis
PG 0859+415
HS 0702+6043
Vega
Gamma Lyrae
Beta Lyrae
R Lyrae
Delta2 Lyrae
Kappa Lyrae
Zeta1 Lyrae
Theta Lyrae
Eta Lyrae
Epsilon Lyrae
Epsilon Lyrae
HD 173780
Lambda Lyrae
16 Lyrae
Mu Lyrae
HD 176051
17 Lyrae
Nu2 Lyrae
Iota Lyrae
HD 176527
Epsilon Lyrae
HD 172044
HD 175740
HD 171301
HD 178233
HD 167965
Delta1 Lyrae
HD 177808
HD 175443
HD 173417
HD 176871
Zeta2 Lyrae
HD 182694
HD 180450
V542 Lyrae
Nu1 Lyrae
19 Lyrae
HD 175635
XY Lyrae
Epsilon Lyrae
HD 176896
HD 173416
HD 173936
HD 174179
HD 177809
HD 182272
HD 171780
HD 174959
HD 168322
V473 Lyrae
V543 Lyrae
HD 179583
V533 Lyrae
HD 174881
HD 172671
HD 181470
HD 175132
HD 181655
HD 168009
HD 181828
HR 7

HD 218559
HD 202418
HD 159517
BP Octantis
HD 58805
HD 203955
HD 165338
Mu2 Octantis
B Octantis
BQ Octantis
Omicron Octantis
HD 142022
HD 212301
CF Octantis
HD 188136
HD 89499
UV Octantis
HD 190290
RV Octantis
EUVE J0317-85.5
AO Octantis
Alpha Ophiuchi
Eta Ophiuchi
Zeta Ophiuchi
Delta Ophiuchi
Beta Ophiuchi
Kappa Ophiuchi
Epsilon Ophiuchi
Theta Ophiuchi
Nu Ophiuchi
72 Ophiuchi
Gamma Ophiuchi
Lambda Ophiuchi
67 Ophiuchi
70 Ophiuchi
44 Ophiuchi
Chi Ophiuchi
45 Ophiuchi
Phi Ophiuchi
36 Ophiuchi
Sigma Ophiuchi
Iota Ophiuchi
Xi Ophiuchi
68 Ophiuchi
Omega Ophiuchi
Psi Ophiuchi
HD 157950
Rho Ophiuchi
Mu Ophiuchi
Upsilon Ophiuchi
20 Ophiuchi
71 Ophiuchi
41 Ophiuchi
Tau Ophiuchi
51 Ophiuchi
66 Ophiuchi
30 Ophiuchi
74 Ophiuchi
58 Ophiuchi
24 Scorpii
66 Herculis
36 Ophiuchi


In [55]:
stars_data_9=get_star_details(Sample9,df)

Omicron Ophiuchi
23 Ophiuchi
Tau Ophiuchi
43 Ophiuchi
HD 158614
37 Ophiuchi
21 Ophiuchi
24 Ophiuchi
HR 6902
73 Ophiuchi
14 Ophiuchi
26 Ophiuchi
12 Ophiuchi
53 Ophiuchi
V2052 Ophiuchi
U Ophiuchi
Rho Ophiuchi
V2213 Ophiuchi
16 Ophiuchi
19 Ophiuchi
V2392 Ophiuchi
V986 Ophiuchi
61 Ophiuchi
V2368 Ophiuchi
33 Scorpii
V1010 Ophiuchi
Y Ophiuchi
2 Sagittarii
V2388 Ophiuchi
V2542 Ophiuchi
29 Ophiuchi
28 Scorpii
36 Ophiuchi
32 Scorpii
V2126 Ophiuchi
X Ophiuchi
V2114 Ophiuchi
V2393 Ophiuchi
52 Ophiuchi
HD 156846
Omicron Ophiuchi
29 Scorpii
HD 154088
HD 155233
38 Ophiuchi
HD 148427
18 Ophiuchi
HD 150433
Gliese 673
HD 157172
HD 149143
HD 170469
HD 164509
HD 171028
HD 152581
HD 159243
Barnard's Star
Wolf 1061
HAT-P-57
WASP-163
CoRoT-6
Gliese 1214
COROT-25
COROT-29
COROT-26
Kepler's Supernova
Rigel
Betelgeuse
Bellatrix
Alnilam
Alnitak
Saiph
Mintaka
Iota Orionis
Pi3 Orionis
Eta Orionis
Meissa
Tau Orionis
Pi4 Orionis
Alnitak
Pi5 Orionis
Sigma Orionis
Omicron2 Orionis
Phi Orionis
Mu Orionis
29 Orionis
32

WASP-18
AD Phoenicis
HIP 65
WASP-29
WASP-105
RV Phoenicis
WASP-96
WASP-5
WASP-4
YZ Phoenicis
WD 2359−434
HE 0107-5240
BPM 30551
Alpha Pictoris
Beta Pictoris
Gamma Pictoris
Delta Pictoris
HD 42540
Eta2 Pictoris
HD 39640
HD 46355
HD 40292
Lambda Pictoris
HD 38871
Eta1 Pictoris
Zeta Pictoris
HD 36553
Iota Pictoris
Nu Pictoris
HD 41214
Mu Pictoris
HD 45557
HD 40733
HD 45984


In [63]:
stars_data_10=get_star_details(Sample10,df)

HD 36734
HD 39937
List of stars in Pictor
HD 35765
Kappa Pictoris
TX Pictoris
HD 40200
HD 50571
AE Pictoris
HD 39110
AK Pictoris
List of stars in Pictor
Theta Pictoris
HD 45796
HD 45461
HD 39547
R Pictoris
HD 38458
HD 39312
Iota Pictoris
HD 37226
HD 37781
HD 44120
SW Pictoris
List of stars in Pictor
HD 45701
HD 45450
HD 34587
HD 40665
S Pictoris
HD 33331
HD 40307
TV Pictoris
HD 41004
HD 41248
Kapteyn's Star
AB Pictoris
RR Pictoris
TW Pictoris
PSR J0437−4715
WISE 0647−6232
Eta Piscium
Gamma Piscium
Omega Piscium
Iota Piscium
Omicron Piscium
Epsilon Piscium
Theta Piscium
Alpha Piscium
30 Piscium
Delta Piscium
Nu Piscium
Beta Piscium
Lambda Piscium
Tau Piscium
33 Piscium
Xi Piscium
Chi Piscium
Phi Piscium
Upsilon Piscium
Mu Piscium
27 Piscium
Kappa Piscium
TX Piscium
47 Piscium
7 Piscium
64 Piscium
29 Piscium
89 Piscium
82 Piscium
Zeta Piscium
Alpha Piscium
91 Piscium
107 Piscium
Psi1 Piscium
Rho Piscium
57 Piscium
41 Piscium
52 Piscium
5 Piscium
2 Piscium
55 Piscium
68 Piscium
20 Piscium

49 G. Sagittarii
70 Sagittarii
71 G. Sagittarii
280 G. Sagittarii
16 G. Sagittarii
HD 165516
26 Sagittarii
HR 7578
297 G. Sagittarii
128 G. Sagittarii
HD 179949
V4199 Sagittarii
6 Sagittarii
206 G. Sagittarii
254 G. Sagittarii
69 G. Sagittarii
109 G. Sagittarii
V4198 Sagittarii
150 G. Sagittarii
30 Sagittarii
17 G. Sagittarii
64 G. Sagittarii
143 G. Sagittarii
282 G. Sagittarii
V4434 Sagittarii
V3894 Sagittarii
68 G. Sagittarii
90 G. Sagittarii
129 G. Sagittarii
263 G. Sagittarii
V3879 Sagittarii
39 G. Sagittarii
167 G. Sagittarii
53 Sagittarii
V4405 Sagittarii
96 G. Sagittarii
127 G. Sagittarii
HD 167665
80 G. Sagittarii
125 G. Sagittarii
133 G. Sagittarii
36 G. Sagittarii
151 G. Sagittarii
V4381 Sagittarii
269 G. Sagittarii
115 G. Sagittarii
103 G. Sagittarii
290 G. Sagittarii
234 G. Sagittarii
V505 Sagittarii
244 G. Sagittarii
9 G. Sagittarii
166 G. Sagittarii
196 G. Sagittarii
V4190 Sagittarii
25 Sagittarii
65 Sagittarii
RY Sagittarii
17 Sagittarii
HD 187085
Chi Sagittarii
HD 18134

In [65]:
stars_data_11=get_star_details(Sample11,df)

OGLE-2003-BLG-235Lb/MOA-2003-BLG-53Lb
SWEEPS-11
MOA-2007-BLG-400Lb
SWEEPS-10
MOA-2007-BLG-192Lb
Pistol Star
WR 102ea
S2 (star)
KW Sagittarii
Antares
Lambda Scorpii
Theta Scorpii
Delta Scorpii
Epsilon Scorpii
Kappa Scorpii
Beta Scorpii
Upsilon Scorpii
Tau Scorpii
Pi Scorpii
Sigma Scorpii
Iota1 Scorpii
Mu1 Scorpii
G Scorpii
Eta Scorpii
Mu2 Scorpii
Zeta2 Scorpii
Rho Scorpii
Omega1 Scorpii
Nu Scorpii
H Scorpii
N Scorpii
Q Scorpii
Omega2 Scorpii
Omicron Scorpii
13 Scorpii
2 Scorpii
1 Scorpii
Zeta1 Scorpii
Xi Scorpii
Iota2 Scorpii
22 Scorpii
HD 161840
HD 146624
V1073 Scorpii
HD 163145
HD 163376
Beta Scorpii
Psi Scorpii
HD 143787
HD 153613
HD 154948
Xi Scorpii
HD 145250
HD 157243
Antares
HD 151804
Chi Scorpii
HD 148688
HD 144690
HD 147513
HD 142165
HR 5907
HD 147628
HD 142990
16 Scorpii
HD 149404
HD 152234
HD 151078
27 Scorpii
18 Scorpii
HD 144987
HD 146836
HD 156098
HD 160668
HD 155806
HD 162587
4 Scorpii
HD 150742
HD 155259
12 Scorpii
HD 159176
11 Scorpii
HD 156293
HD 152408
HD 148247
HD 14

Sterope (star)
70 Tauri
2 G. Tauri
26 Tauri
55 G. Tauri
9 G. Tauri
37 G. Tauri
SZ Tauri
9 Tauri
101 Tauri
HD 24496
55 Tauri
HD 24040
HD 32963
HD 37124
Alcyone (star)
HD 28678
RV Tauri
HD 283668
T Tauri
HD 284149
HD 286123
Gliese 176
List of exoplanets discovered in 2013
HD 283869
V830 Tauri
WASP-152
CL Tauri
CI Tauri
FW Tauri
Crab Pulsar
2MASS J04414489+2301513
Alpha Telescopii
Zeta Telescopii
Epsilon Telescopii
Lambda Telescopii
Iota Telescopii
Delta1 Telescopii
Xi Telescopii
Eta Telescopii
Delta2 Telescopii
Rho Telescopii
Kappa Telescopii
Nu Telescopii
HR 6819
HD 179886
HD 169405
HD 174387
PW Telescopii
HD 191829
HD 170069
HD 182509
HD 183552
HD 187420
HD 173791
HD 171819
HD 187086
HD 176664
HD 177406
HD 167257
HD 166006
HD 178845
HD 192886
HD 182893
HD 174500
HD 174474
HD 189080
HD 170873
HD 173047
QQ Telescopii
HD 185075
HD 190422


In [71]:
stars_data_12=get_star_details(Sample12,df)

HD 193307
HD 186756
HD 193002
HD 192827
HD 177365
Mu Telescopii
HD 174430
HD 180134
HD 181295
HD 177693
HD 168871
RX Telescopii
HD 190879
HD 172223
HD 190580
HD 191732
HD 187421
HD 181327
BL Telescopii
HD 191760
HO Telescopii
PZ Telescopii
HD 176387
PV Telescopii
RS Telescopii
KELT-10
RR Telescopii
Gliese 754
KK Telescopii
EC 20058-5234
QS Telescopii
Beta Trianguli
Alpha Trianguli
Gamma Trianguli
Delta Trianguli
Iota Trianguli
14 Trianguli
7 Trianguli
10 Trianguli
12 Trianguli
15 Trianguli
R Trianguli
Epsilon Trianguli
11 Trianguli
HD 10390
HD 11007
HD 11928
HD 15755
13 Trianguli
HD 10348
HD 16187
HD 16220
5 Trianguli
HD 15464
HD 10588
HD 13747
HD 9298
1 Trianguli
HD 13189
HD 9446
XX Trianguli
X Trianguli
V Trianguli
RV Trianguli
WASP-56
RW Trianguli
HAT-P-38
AI Trianguli
Var 83
Var B (star)
Var C (star)
Var 19
M33 X-7
Var A
Var 2
LSPM J0207+3331
Alpha Trianguli Australis
Beta Trianguli Australis
Gamma Trianguli Australis
Delta Trianguli Australis
Epsilon Trianguli Australis
Zeta Trian

252 G. Virginis
37 Virginis
183 G. Virginis
5 G. Virginis
56 G. Virginis
257 G. Virginis
72 Virginis
41 G. Virginis
34 Virginis
38 Virginis
9 G. Virginis
230 G. Virginis
271 G. Virginis
1 G. Virginis
19 G. Virginis
153 G. Virginis
85 Virginis
239 G. Virginis
104 Virginis
261 G. Virginis
57 G. Virginis
199 G. Virginis
135 G. Virginis
3 G. Virginis
27 Virginis
231 G. Virginis
FT Virginis
27 G. Virginis
209 G. Virginis
41 Virginis
HD 126053
54 Virginis
134 G. Virginis
85 G. Virginis
213 G. Virginis
20 Virginis
208 G. Virginis
IQ Virginis
EP Virginis
26 G. Virginis
112 G. Virginis
207 G. Virginis
68 G. Virginis
98 G. Virginis
185 G. Virginis
210 G. Virginis
51 G. Virginis
129 G. Virginis
204 G. Virginis
225 G. Virginis
33 G. Virginis
29 G. Virginis
211 G. Virginis
FS Virginis
35 Virginis
150 G. Virginis
221 G. Virginis
240 G. Virginis
141 G. Virginis
229 G. Virginis
91 G. Virginis
96 Virginis
254 G. Virginis
17 Virginis
49 G. Virginis
233 G. Virginis
235 G. Virginis
88 G. Virginis
20 G. Vi

In [21]:
stars_data.to_excel("stars_1.xlsx")

In [72]:
frames=[stars_data_1,stars_data_2,stars_data_3,stars_data_4,stars_data_5,stars_data_6,stars_data_7,
        stars_data_8,stars_data_9,stars_data_10,stars_data_11,stars_data_12]

In [73]:
final_data_frame=pd.concat(frames,ignore_index=True)

In [74]:
final_data_frame

,Name,Constellation,Right ascension,Declination,Apparent magnitude (V),Spectral type,U−B color index,B−V color index,V−R color index,R−I color index,...,Metallicity [Fe/H],Rotational velocity (v sin i),Age,Period (P),Semi-major axis (a),Eccentricity (e),Inclination (i),Longitude of the node (Ω),Periastron epoch (T),Argument of periastron (ω)(secondary)
0,Alpha Andromedae,Andromeda,00h 08m 23.25988s,+29° 05′ 25.5520″,2.06 (2.22 + 4.21),B8IVpMnHg,−0.46,−0.11,,−0.10,...,,52 km/s,60 Myr,96.7015 ± 0.0044 days,24.0 ± 0.13 mas,0.535 ± 0.0046,105.6 ± 0.23°,284.4 ± 0.21°,MJD 47374.563 ± 0.095,257.4 ± 0.31°
1,Beta Andromedae,Andromeda,01h 09m 43.92388s,+35° 37′ 14.0075″,2.05 (2.01 to 2.10),M0 III,+1.96,+1.57,0.9,+1.00,...,−0.05 dex,7.2 km/s,,,,,,,,
2,Gamma Andromedae,Andromeda,02h 03m 53.95229s,+42° 19′ 47.0223″,2.27,K2+IIb,+1.58,+1.37,,+0.68,...,−0.06 dex,< 17 km/s,6.5±0.1 Myr,"4,748 yr",,,,,,
3,Delta Andromedae,Andromeda,00h 39m 19.67518s,+30° 51′ 39.6783″,3.28,K3 III + K4 ± 2,+1.48,+1.28,,+0.66,...,0.04 dex,6.5 km/s,3.2 Gyr,"15,000 d",,0.34±0.14,,,2415568 JD,
4,Andromeda Galaxy,Andromeda,00h 42m 44.3s,+41° 16′ 9″,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093,GS 2000+25,Vulpecula,20h 02m 49.58s,+25° 14′ 11.3″,18.2,Black hole + K3-6 V,,,,,...,,,,8.26 hr,,,,,,
4094,CK Vulpeculae,Vulpecula,19h 47m 38.0s,+27° 18′ 48″,max 2.6,,,0.7,,,...,,,,,,,,,,
4095,PSR B1919+21,Vulpecula,19h 21m 44.815s,+21° 53′ 02.25″,,,,,,,...,,,16 Myr,,,,,,,
4096,PSR B1937+21,Vulpecula,19h 39m 38.560210s,+21° 34′ 59.14166″,,Pulsar,,,,,...,,,2.29 × 108 years,,,,,,,


In [76]:
final_data_frame.to_excel("stars_final.xlsx")